In [1]:
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import numpy as np
import csv

### Se carga el archivo del set de entrenamiento

In [2]:
with open('../input/train.csv', 'r') as trainFile :
    trainReader = csv.reader(trainFile, delimiter=',', quotechar='"')
    trainFile.next()
    trainData = []
    for row in trainReader :
        trainTarget = int(row[0])
        rowData = np.zeros(28*28)
        for i in xrange(28*28) :
            rowData[i] = row[i+1]
                
        trainData.append((trainTarget,rowData))
                

In [3]:
trainDataRDD = sc.parallelize(trainData,64)

## A continuacion se filtran los datos para que sean procesados por clase

In [4]:
trainDataByClasses = []
for i in xrange(10):
    trainDataByClassesRDD = trainDataRDD.filter(lambda tupla : tupla[0] == i)
    trainDataByClasses.append(trainDataByClassesRDD)


### Se calcula el promedio de densidad correspondiente a cada clase

In [5]:
def weightOfImage(row):
    summarize = 0
    i,j = 0,0
    for value in row :
        if value != 0 :
            j+=1
        i+=1
    return float(j)/float(i)


#### Se crea una tupla del tipo (value,array,weight)

In [6]:
weightImages = []
for i in xrange(10):
    rddIndex = trainDataByClasses[i]
    rddAlgo = rddIndex.map(lambda tupla: (tupla[0],tupla[1],weightOfImage(tupla[1])))
    weightImages.append(rddAlgo)
    


### Se reduce un %70 la cantidad de datos de nuestro set de entrenamiento, en base a la densidad de la imagen.

In [7]:
reducedDataSet = []
for i in xrange(10) :
    rowRDD = weightImages[i]
    lenRow = rowRDD.count()
    reducedDataSet.append(rowRDD.takeOrdered(int(lenRow*0.70),lambda thrupla: thrupla[2]))

In [106]:
listReducedDataSet = []
for i in xrange(10):
    for j in xrange(len(reducedDataSet[i])):
        listReducedDataSet.append(reducedDataSet[i][j])

29394 29394


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


ValueError: Found arrays with inconsistent numbers of samples: [    1 29394]

[0.12372448979591837, 0.10076530612244898]

[]

[0.22704081632653061, 0.23341836734693877]

[0.1441326530612245, 0.21556122448979592]

[0.20918367346938777, 0.29846938775510207]

[0.23214285714285715, 0.18112244897959184]

[0.14540816326530612, 0.1211734693877551]

[0.10714285714285714, 0.11224489795918367]

[0.1989795918367347, 0.16454081632653061]

2512.7999999999997